In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['pytorch_geometric_test.pkl',
 'laplaction_matrix_test.pkl',
 'pytorch_geometric_train.pkl',
 'laplaction_matrix_train.pkl']

In [5]:
mytrial = []

In [ ]:
"""
y=scalar_coupling_constant, type=0
374
{'columns': ['atom_1_bond_lengths_std', 'yukawa_N.x', 'dist_C_2_y', 'dist_H_1_x', 'molecule_type_dist_mean_diff', 'inv_distPR', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_y', 'dist_N_1_y', 'dist_O_1_x', 'eem_1', 'tertiary_angle_5', 'dist_to_type_std', 'tertiary_distance_3', 'cos_center1', 'tertiary_angle_3', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_H_2_x', 'mean_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'tertiary_angle_4', 'dist_C_3_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_4_y', 'dist_to_type_0_mean', 'molecule_atom_index_1_dist_min_diff', 'distC0', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'adN1', 'tertiary_distance_10', 'tertiary_atom_0', 'dist_to_type_mean', 'dist_C_1_x', 'adC1', 'tertiary_distance_1', 'molecule_dist_max', 'tertiary_distance_6', 'inv_dist1E', 'cos_f0', 'tertiary_distance_5', 'inv_dist1R', 'cos_center0', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'cos_f1', 'yukawa_H.x', 'tertiary_distance_2', 'coulomb_H.y', 'atom_index_1_n_cycle', 'inv_dist1', 'tertiary_distance_0', 'dist_C_0_y', 'tertiary_atom_1', 'adC3', 'dist_H_3_y', 'dist_H_4_y', 'dist_C_1_y', 'dist_O_1_y', 'atom_1_n_bonds', 'dist_xyz', 'distC1', 'dist_N_0_x', 'adC2', 'atom_index_farthest_0', 'dist_H_3_x', 'vander_C.y', 'inv_dist0R', 'dist_H_0_x', 'dist_C_2_x', 'dist_C_3_x', 'dist_H_2_y', 'dist_C_4_x', 'tertiary_distance_7', 'tertiary_angle_1', 'dist_H_0_y', 'vander_N.x', 'eem_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=1
513
{'columns': ['bond_atom', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'inv_dist1', 'dist_O_0_y', 'inv_distPR', 'dist_H_1_x', 'tertiary_distance_0', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'dist_C_0_y', 'dist_N_1_y', 'linkN', 'adC3', 'dist_H_3_y', 'cos_center0_center1', 'distance_center1', 'dist_C_1_y', 'tertiary_distance_3', 'min_molecule_atom_1_dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'adC2', 'molecule_atom_index_1_dist_max_diff', 'dist_C_0_x', 'dist_C_3_y', 'molecule_atom_index_0_dist_max_div', 'dist_to_type_0_mean', 'inv_dist0R', 'tertiary_distance_4', 'dist_O_0_x', 'vander_H.x', 'tertiary_angle_0', 'dist_H_0_x', 'tertiary_atom_0', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'coulomb_H.x', 'distance_center0', 'cos_f0', 'tertiary_distance_5', 'inv_dist1R', 'inv_distP', 'dist_N_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'tertiary_distance_2', 'dist_H_0_y', 'vander_N.x', 'yukawa_H.x', 'eem_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=2
740
{'columns': ['bond_atom', 'atom_1_bond_lengths_std', 'dist_H_1_x', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_y', 'tertiary_atom_2', 'dist_O_1_x', 'eem_1', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_H_1_y', 'dist_N_0_y', 'dist_C_0_x', 'tertiary_angle_4', 'molecule_atom_index_0_dist_max_div', 'cos_c0_c1', 'molecule_atom_index_1_dist_min_diff', 'distC0', 'dist_O_0_x', 'yukawa_O.y', 'vander_H.x', 'molecule_type_dist_max', 'tertiary_angle_0', 'adN1', 'dist_to_type_1_mean', 'tertiary_atom_0', 'dist_to_type_mean', 'dist_C_1_x', 'adC1', 'tertiary_distance_1', 'tertiary_distance_6', 'cos_f0', 'inv_dist1R', 'cos_c0', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'tertiary_distance_2', 'inv_dist1', 'cos_c1', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'adC3', 'dist_C_1_y', 'min_molecule_atom_0_dist_xyz', 'atom_1_n_bonds', 'atom_0_bond_lengths_max', 'dist_N_0_x', 'adC2', 'molecule_atom_index_1_dist_min_div', 'yukawa_O.x', 'inv_dist0R', 'dist_H_0_x', 'vander_O.y', 'dist_C_2_x', 'dist_C_3_x', 'dist_H_2_y', 'inv_dist0', 'tertiary_atom_4', 'inv_distP', 'atom_1_bond_lengths_min', 'tertiary_atom_3', 'dist_C_4_x', 'tertiary_angle_1', 'dist_H_0_y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
1050
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
1126
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
1202
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=6
360
{'columns': ['tertiary_atom_2', 'distH0', 'tertiary_angle_4', 'dist_C_3_x', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_atom_1', 'molecule_atom_index_1_dist_std_div', 'adH1', 'dist_C_3_y', 'inv_dist1', 'inv_distPR', 'dist_H_2_y', 'dist_C_1_y', 'cos_c0', 'dist_C_2_x', 'cos_f1', 'dist_to_type_mean', 'cos_c1', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_diff', 'inv_dist0', 'dist_C_1_x', 'dist_to_type_0_mean', 'dist_H_1_x', 'dist_O_0_x', 'dist_xyz', 'molecule_atom_index_0_dist_max_diff', 'tertiary_distance_3', 'adC3', 'adH2', 'molecule_atom_index_0_dist_max_div', 'adC2', 'dist_C_2_y', 'dist_N_0_y', 'max_distance_y', 'tertiary_angle_1', 'dist_H_0_y', 'tertiary_distance_1', 'tertiary_angle_2', 'molecule_atom_index_0_dist_min_diff', 'inv_dist0R', 'dist_to_type_std', 'dist_to_type_1_mean', 'cos_f0_f1', 'tertiary_atom_0', 'link0', 'distance_c1', 'dist_O_1_x', 'dist_H_1_y', 'molecule_atom_index_1_dist_max_diff', 'adN1', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_H_0_x', 'mean_molecule_atom_0_dist_xyz', 'tertiary_distance_4', 'cos_c0_c1', 'cos_f0', 'dist_C_0_x', 'linkM0', 'tertiary_distance_2', 'dist_C_0_y', 'molecule_atom_index_1_dist_max_div', 'tertiary_angle_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
527
{'columns': ['tertiary_distance_3', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_x', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_atom_1', 'adC2', 'dist_C_3_y', 'dist_C_2_y', 'dist_N_0_y', 'tertiary_angle_2', 'tertiary_distance_1', 'tertiary_angle_1', 'dist_H_0_y', 'inv_distPR', 'molecule_atom_index_0_dist_min_diff', 'atom_1_bond_lengths_mean', 'dist_to_type_1_mean', 'cos_c0', 'dist_C_2_x', 'atom_1_bond_lengths_std', 'atom_index_1_cycle_size_mean', 'cos_c1', 'molecule_atom_index_0_dist_std_diff', 'tertiary_angle_3', 'dist_to_type_mean', 'max_molecule_atom_0_dist_xyz', 'dist_H_1_y', 'atom_0_bond_lengths_max', 'dist_C_4_x', 'dist_N_0_x', 'adC1', 'dist_H_0_x', 'tertiary_distance_4', 'dist_C_1_x', 'cos_c0_c1', 'atom_1_n_bonds', 'cos_f0', 'dist_O_0_x', 'dist_H_1_x', 'bond_atom', 'dist_C_0_x', 'tertiary_distance_2', 'dist_C_0_y', 'atom_1_bond_lengths_max', 'yukawa_H.y', 'molecule_atom_index_1_dist_max_div', 'tertiary_angle_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 3}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
"""

In [6]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in np.arange(6,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 300000 if df_train.shape[0] > 300000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'GroupKFold',
                'init':{
                    'n_splits': 3,
#                     'shuffle': True,
#                     'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=scalar_coupling_constant, type=6
360
{'columns': ['tertiary_atom_2', 'distH0', 'tertiary_angle_4', 'dist_C_3_x', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_atom_1', 'molecule_atom_index_1_dist_std_div', 'adH1', 'dist_C_3_y', 'inv_dist1', 'inv_distPR', 'dist_H_2_y', 'dist_C_1_y', 'cos_c0', 'dist_C_2_x', 'cos_f1', 'dist_to_type_mean', 'cos_c1', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_diff', 'inv_dist0', 'dist_C_1_x', 'dist_to_type_0_mean', 'dist_H_1_x', 'dist_O_0_x', 'dist_xyz', 'molecule_atom_index_0_dist_max_diff', 'tertiary_distance_3', 'adC3', 'adH2', 'molecule_atom_index_0_dist_max_div', 'adC2', 'dist_C_2_y', 'dist_N_0_y', 'max_distance_y', 'tertiary_angle_1', 'dist_H_0_y', 'tertiary_distance_1', 'tertiary_angle_2', 'molecule_atom_index_0_dist_min_diff', 'inv_dist0R', 'dist_to_type_std', 'dist_to_type_1_mean', 'cos_f0_f1', 'tertiary_atom_0', 'link0', 'distance_c1', 'dist_O_1_x', 'dist_H_1_y', 'molecule_atom_index_1_dis

In [8]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [13]:
columns_list = [
{'columns': ['atom_1_bond_lengths_std', 'yukawa_N.x', 'dist_C_2_y', 'dist_H_1_x', 'molecule_type_dist_mean_diff', 'inv_distPR', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_y', 'dist_N_1_y', 'dist_O_1_x', 'eem_1', 'tertiary_angle_5', 'dist_to_type_std', 'tertiary_distance_3', 'cos_center1', 'tertiary_angle_3', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_H_2_x', 'mean_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'tertiary_angle_4', 'dist_C_3_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_4_y', 'dist_to_type_0_mean', 'molecule_atom_index_1_dist_min_diff', 'distC0', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'adN1', 'tertiary_distance_10', 'tertiary_atom_0', 'dist_to_type_mean', 'dist_C_1_x', 'adC1', 'tertiary_distance_1', 'molecule_dist_max', 'tertiary_distance_6', 'inv_dist1E', 'cos_f0', 'tertiary_distance_5', 'inv_dist1R', 'cos_center0', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'cos_f1', 'yukawa_H.x', 'tertiary_distance_2', 'coulomb_H.y', 'atom_index_1_n_cycle', 'inv_dist1', 'tertiary_distance_0', 'dist_C_0_y', 'tertiary_atom_1', 'adC3', 'dist_H_3_y', 'dist_H_4_y', 'dist_C_1_y', 'dist_O_1_y', 'atom_1_n_bonds', 'dist_xyz', 'distC1', 'dist_N_0_x', 'adC2', 'atom_index_farthest_0', 'dist_H_3_x', 'vander_C.y', 'inv_dist0R', 'dist_H_0_x', 'dist_C_2_x', 'dist_C_3_x', 'dist_H_2_y', 'dist_C_4_x', 'tertiary_distance_7', 'tertiary_angle_1', 'dist_H_0_y', 'vander_N.x', 'eem_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['bond_atom', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'inv_dist1', 'dist_O_0_y', 'inv_distPR', 'dist_H_1_x', 'tertiary_distance_0', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'dist_C_0_y', 'dist_N_1_y', 'linkN', 'adC3', 'dist_H_3_y', 'cos_center0_center1', 'distance_center1', 'dist_C_1_y', 'tertiary_distance_3', 'min_molecule_atom_1_dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'adC2', 'molecule_atom_index_1_dist_max_diff', 'dist_C_0_x', 'dist_C_3_y', 'molecule_atom_index_0_dist_max_div', 'dist_to_type_0_mean', 'inv_dist0R', 'tertiary_distance_4', 'dist_O_0_x', 'vander_H.x', 'tertiary_angle_0', 'dist_H_0_x', 'tertiary_atom_0', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'coulomb_H.x', 'distance_center0', 'cos_f0', 'tertiary_distance_5', 'inv_dist1R', 'inv_distP', 'dist_N_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'tertiary_distance_2', 'dist_H_0_y', 'vander_N.x', 'yukawa_H.x', 'eem_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['bond_atom', 'atom_1_bond_lengths_std', 'dist_H_1_x', 'max_molecule_atom_0_dist_xyz', 'dist_O_0_y', 'tertiary_atom_2', 'dist_O_1_x', 'eem_1', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_H_1_y', 'dist_N_0_y', 'dist_C_0_x', 'tertiary_angle_4', 'molecule_atom_index_0_dist_max_div', 'cos_c0_c1', 'molecule_atom_index_1_dist_min_diff', 'distC0', 'dist_O_0_x', 'yukawa_O.y', 'vander_H.x', 'molecule_type_dist_max', 'tertiary_angle_0', 'adN1', 'dist_to_type_1_mean', 'tertiary_atom_0', 'dist_to_type_mean', 'dist_C_1_x', 'adC1', 'tertiary_distance_1', 'tertiary_distance_6', 'cos_f0', 'inv_dist1R', 'cos_c0', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_mean', 'atom_index_1_cycle_size_mean', 'tertiary_angle_2', 'tertiary_distance_2', 'inv_dist1', 'cos_c1', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'adC3', 'dist_C_1_y', 'min_molecule_atom_0_dist_xyz', 'atom_1_n_bonds', 'atom_0_bond_lengths_max', 'dist_N_0_x', 'adC2', 'molecule_atom_index_1_dist_min_div', 'yukawa_O.x', 'inv_dist0R', 'dist_H_0_x', 'vander_O.y', 'dist_C_2_x', 'dist_C_3_x', 'dist_H_2_y', 'inv_dist0', 'tertiary_atom_4', 'inv_distP', 'atom_1_bond_lengths_min', 'tertiary_atom_3', 'dist_C_4_x', 'tertiary_angle_1', 'dist_H_0_y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_2_y', 'dist_H_1_x', 'tertiary_atom_2', 'cos_c1', 'dist_O_0_y', 'dist_C_0_y', 'tertiary_atom_1', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_index_0_dist_min_div', 'adC3', 'dist_H_3_y', 'dist_C_1_y', 'tertiary_distance_3', 'tertiary_angle_3', 'dist_xyz', 'dist_N_0_y', 'dist_H_1_y', 'yukawa_H.y', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'adC2', 'dist_C_0_x', 'dist_C_3_y', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'dist_to_type_0_mean', 'tertiary_distance_4', 'dist_O_0_x', 'molecule_dist_min', 'molecule_type_dist_max', 'vander_H.x', 'tertiary_angle_0', 'dist_to_type_1_mean', 'dist_H_0_x', 'adN1', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_1_x', 'tertiary_distance_1', 'dist_H_2_y', 'link0', 'inv_dist0', 'cos_f0_f1', 'cos_f0', 'max_distance_y', 'cos_c0', 'inv_distP', 'tertiary_angle_1', 'cos_f1', 'tertiary_distance_2', 'dist_H_0_y', 'tertiary_angle_2', 'yukawa_H.x', 'inv_distPE', 'coulomb_H.y'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_atom_2', 'distH0', 'tertiary_angle_4', 'dist_C_3_x', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_atom_1', 'molecule_atom_index_1_dist_std_div', 'adH1', 'dist_C_3_y', 'inv_dist1', 'inv_distPR', 'dist_H_2_y', 'dist_C_1_y', 'cos_c0', 'dist_C_2_x', 'cos_f1', 'dist_to_type_mean', 'cos_c1', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_diff', 'inv_dist0', 'dist_C_1_x', 'dist_to_type_0_mean', 'dist_H_1_x', 'dist_O_0_x', 'dist_xyz', 'molecule_atom_index_0_dist_max_diff', 'tertiary_distance_3', 'adC3', 'adH2', 'molecule_atom_index_0_dist_max_div', 'adC2', 'dist_C_2_y', 'dist_N_0_y', 'max_distance_y', 'tertiary_angle_1', 'dist_H_0_y', 'tertiary_distance_1', 'tertiary_angle_2', 'molecule_atom_index_0_dist_min_diff', 'inv_dist0R', 'dist_to_type_std', 'dist_to_type_1_mean', 'cos_f0_f1', 'tertiary_atom_0', 'link0', 'distance_c1', 'dist_O_1_x', 'dist_H_1_y', 'molecule_atom_index_1_dist_max_diff', 'adN1', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_H_0_x', 'mean_molecule_atom_0_dist_xyz', 'tertiary_distance_4', 'cos_c0_c1', 'cos_f0', 'dist_C_0_x', 'linkM0', 'tertiary_distance_2', 'dist_C_0_y', 'molecule_atom_index_1_dist_max_div', 'tertiary_angle_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_3', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_x', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_atom_1', 'adC2', 'dist_C_3_y', 'dist_C_2_y', 'dist_N_0_y', 'tertiary_angle_2', 'tertiary_distance_1', 'tertiary_angle_1', 'dist_H_0_y', 'inv_distPR', 'molecule_atom_index_0_dist_min_diff', 'atom_1_bond_lengths_mean', 'dist_to_type_1_mean', 'cos_c0', 'dist_C_2_x', 'atom_1_bond_lengths_std', 'atom_index_1_cycle_size_mean', 'cos_c1', 'molecule_atom_index_0_dist_std_diff', 'tertiary_angle_3', 'dist_to_type_mean', 'max_molecule_atom_0_dist_xyz', 'dist_H_1_y', 'atom_0_bond_lengths_max', 'dist_C_4_x', 'dist_N_0_x', 'adC1', 'dist_H_0_x', 'tertiary_distance_4', 'dist_C_1_x', 'cos_c0_c1', 'atom_1_n_bonds', 'cos_f0', 'dist_O_0_x', 'dist_H_1_x', 'bond_atom', 'dist_C_0_x', 'tertiary_distance_2', 'dist_C_0_y', 'atom_1_bond_lengths_max', 'yukawa_H.y', 'molecule_atom_index_1_dist_max_div', 'tertiary_angle_0'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [14]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [15]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-30 00:58:27.107962,modeling for 0,87,0.814762,0.935848,0.121086,modeling for 0
1,2019-07-30 00:58:44.629557,modeling for 3,56,0.161362,0.199745,0.038383,modeling for 3
2,2019-07-30 00:58:47.396305,modeling for 1,59,0.285739,0.438729,0.152991,modeling for 1
3,2019-07-30 00:58:54.627443,modeling for 4,56,0.154612,0.237577,0.082965,modeling for 4
4,2019-07-30 00:59:56.406682,modeling for 2,68,0.364386,0.396686,0.032300,modeling for 2
5,2019-07-30 01:00:28.870033,modeling for 6,66,0.191407,0.219717,0.028310,modeling for 6
6,2019-07-30 01:01:44.757529,modeling for 5,56,0.430255,0.459736,0.029481,modeling for 5
7,2019-07-30 01:01:53.112985,modeling for 7,48,0.101460,0.142940,0.041480,modeling for 7


In [16]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.44807359104748623
0 0.9358483862212079
3 0.19974463780640106
1 0.43872980374242654
4 0.23757730499893367
2 0.3966864996961603
6 0.21971668375988143
5 0.4597358535700776
7 0.14293951255130133


In [18]:
idx=0.448
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)